# cDCGAN (conditional Deep Convolutional GAN)

**Author:** Teebone Ding

**Date:** 2019 Feb

Based on DCGAN (Radford, Metz, 2016) [Paper Link](https://arxiv.org/pdf/1511.06434.pdf)

Some useful refs while I implementing my cDCGAN:
* keras GAN [Link](https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py)
* cDCGAN in tensorflow version [Link](https://github.com/znxlwm/tensorflow-MNIST-cGAN-cDCGAN)
* cDCGAN using keras [Link](https://medium.com/@utk.is.here/training-a-conditional-dc-gan-on-cifar-10-fce88395d610)

Using **CIFAR 10 dataset** as playground.

My SW/HW setting:
* ASUS nVidia GTX 1060 3GB
* RAM: 16GB
* CPU: Intel Core i5-7400 3.00GHz
* Ubuntu 16.04 LTS

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

1.12.0


## Load CIFAR 10 data and data explore

In [3]:
# Load CIFAR 10 data
cifar10_dataset = keras.datasets.cifar10
(x_train, y_train), (_, _) = cifar10_dataset.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [5]:
# Ref: https://github.com/EN10/CIFAR
y_cat = {
    0 : "airplane",
    1 : "automobile",
    2 : "bird",
    3 : "cat",
    4 : "deer",
    5 : "dog",
    6 : "frog",
    7 : "horse",
    8 : "ship",
    9 : "truck"
}

In [6]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)

In [7]:
print(y_train_one_hot.shape)
print(y_train_one_hot[:10])

(50000, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [8]:
y_train[:10]

array([[6],
       [9],
       [9],
       [4],
       [1],
       [1],
       [2],
       [7],
       [8],
       [3]], dtype=uint8)

In [9]:
x_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

# Preprocess training data

In [10]:
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
#x_train = x_train.astype(np.float32)/255.0

In [11]:
x_train.shape

(50000, 32, 32, 3)

## Build Generator and Discriminator
Try CNN layers to build GAN (cDCGAN).

In [12]:
from tensorflow.keras.layers import Input, Dense, Concatenate, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, ReLU, UpSampling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [13]:
z = Input(shape=(100,))    # Normal distribution noise input
x = Input(shape=(32,32,3)) # image, by real dataset or generator
y = Input(shape=(10,))     # label, MNIST one hot encoded with 10 labels

In [14]:
def build_generator(z,y):
    layer = Concatenate()([z,y])
    
    layer = Dense((4*4*256))(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Reshape((4,4,256))(layer)
    
    layer = Conv2DTranspose(128,5,strides=2,padding='same')(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)

    layer = Conv2DTranspose(64,5,strides=2, activation='relu',padding='same')(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)

    layer = Conv2DTranspose(3,5,strides=2, activation='tanh',padding='same')(layer)
    
    model = Model([z,y],layer)

    return model,layer

G,G_out = build_generator(z,y)
G.summary() # not compiled, combined with discriminator and compile.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 110)          0           input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 4096)         454656      concatenate[0][0]                
__________

In [15]:
def build_discriminator(x,y):
    layer = Conv2D(32,3,strides=2,padding="same")(x)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    
    layer = Conv2D(64,3,strides=2,padding="same")(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    
    layer = Conv2D(128,3,strides=2,padding="same")(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    
    layer = Flatten()(layer)
    layer = Concatenate()([layer, y])
    layer = Dense(256,activation='relu')(layer)
    
    layer = Dense(1,activation='sigmoid')(layer)
    
    model = Model([x,y],layer)
    return model, layer

D, D_out = build_discriminator(x,y)
#D.summary()

In [16]:
# While training....
# 1. train discriminator first
# 2. train a combined (G+D) model, with D is not trainable (only train G)
# 3. calculate loss value and save weights

In [17]:
# Compile Discriminator model
#sgd = SGD(lr=0.1, momentum=0.5 ,decay= 1.00004)
adam = Adam(0.0002, 0.5)
D.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
D.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 32)   896         input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)       (None, 16, 16, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 16, 16, 32)   0           leaky_re_lu_3[0][0]              
__________________________________________________________________________________________________
conv2d_1 (

In [18]:
# Compile combined G+D model with D is not trainable
img = G([z,y])
D.trainable = False
valid = D([img,y])
C = Model([z,y], valid)
C.summary()
C.compile(optimizer=adam,loss='binary_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 32, 32, 3)    1500803     input_1[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            621377      model[1][0]                      
          

# Train conditional GAN

In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

In [20]:
# dataset: x_train, y_train_one_hot

In [21]:
# Randomly generate images from Generator network
def sample_gen_imgs(epoch):
    gen_labels = np.repeat(np.arange(10),10)
    gen_labels_cat = to_categorical(gen_labels, num_classes=10)
    noises = np.random.normal(0,1,(100,100))
    gen_imgs = G.predict([noises,gen_labels_cat])
    gen_imgs = 0.5 * gen_imgs + 0.5

    r,c = 10,10
    fig, axs = plt.subplots(r, c, figsize=(20, 20))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,:])
            axs[i,j].set_title("Cat: %s" % y_cat[gen_labels[cnt]])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/gen_cifar10_%d.png"%epoch)
    plt.close()

In [22]:
import pdb
import datetime
def train(epochs = 10 ,batch_size = 128): 
    real_label = np.ones((batch_size,1))
    fake_label = np.zeros((batch_size,1))
    
    for epoch in range(epochs):
        epoch += 1
        for step in range(int(x_train.shape[0]/batch_size)):
            # Train Discriminator once
            # sample real data from training dataset
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs, labels = x_train[idx], y_train_one_hot[idx]
            # sample noises from normal dist.
            # Somehow I tried uniform dist but cannot trained very well.
            noises = np.random.normal(0,1,(batch_size,100))
            # generate fake images
            gen_imgs = G.predict([noises,labels])
            #pdb.set_trace()
            # Train Discriminator
            d_real_loss = D.train_on_batch([imgs,labels], real_label) 
            d_fake_loss = D.train_on_batch([gen_imgs,labels],fake_label)
            d_loss = 0.5*np.add(d_real_loss,d_fake_loss)
           
            # Train Combined model (generator)
            gen_labels = to_categorical(np.random.randint(0,10, batch_size),num_classes=10)
            # train generator
            g_loss = C.train_on_batch([noises,gen_labels], real_label)
            print("[%s] D-Loss value: %.5f Acc: %.5f G-Loss value: %.5f in epoch: %d"%(datetime.datetime.now(),d_loss[0],d_loss[1], g_loss, epoch),end='\r')
        
        if epoch % 5 == 0:
            print("[%s] D-Loss value: %.5f Acc: %.5f G-Loss value: %.5f in epoch: %d"%(datetime.datetime.now(),d_loss[0],d_loss[1], g_loss, epoch))
            sample_gen_imgs(epoch)
            G.save("models/G_cifar10_%d.h5"%epoch)
            C.save("models/C_cifar10_%d.h5"%epoch)
            D.save("models/D_cifar10_%d.h5"%epoch)

In [23]:
train(epochs=200, batch_size = 128)

[2019-02-17 01:34:31.233285] D-Loss value: 0.65242 Acc: 0.55859 G-Loss value: 0.81228 in epoch: 5
[2019-02-17 01:36:47.721813] D-Loss value: 0.64420 Acc: 0.59375 G-Loss value: 0.86993 in epoch: 10
[2019-02-17 01:39:02.077943] D-Loss value: 0.61034 Acc: 0.65234 G-Loss value: 0.82328 in epoch: 15
[2019-02-17 01:41:17.647435] D-Loss value: 0.63431 Acc: 0.62500 G-Loss value: 0.99098 in epoch: 20
[2019-02-17 01:43:32.604386] D-Loss value: 0.61807 Acc: 0.67969 G-Loss value: 0.97781 in epoch: 25
[2019-02-17 01:45:47.201714] D-Loss value: 0.59697 Acc: 0.66406 G-Loss value: 1.03059 in epoch: 30
[2019-02-17 01:48:00.784945] D-Loss value: 0.56278 Acc: 0.69531 G-Loss value: 1.31992 in epoch: 35
[2019-02-17 01:50:13.981386] D-Loss value: 0.57437 Acc: 0.67969 G-Loss value: 1.16380 in epoch: 40
[2019-02-17 01:52:26.423468] D-Loss value: 0.57058 Acc: 0.69531 G-Loss value: 1.29966 in epoch: 45
[2019-02-17 01:54:39.153861] D-Loss value: 0.51956 Acc: 0.75000 G-Loss value: 1.34080 in epoch: 50
[2019-02-17

## Generated images after 200 epochs
![alt text](images/gen_cifar10_200.png "Gen image 200 epochs")
